In [1]:
import pandas as pd

In [2]:

from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select


import pandas as pd

driver_path = '/Users/zacharywong/Downloads/chromedriver'
service = Service(driver_path)

url = 'https://www.shazam.com/charts/top-200/united-states'

driver = webdriver.Chrome(service = service)
driver.get(url)
xpath = '//select[@data-shz-type="city"]'
#id = 'charts.ue-download-csv'

element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
try:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)
    
except:
    cities = driver.find_element(By.XPATH, xpath)
    dropdown = Select(cities)




In [3]:
#cities
citylist = []

for city in dropdown.options:
    citynormalized = city.text.replace(',', '').replace(' ', '-')
    citylist.append(citynormalized)
citylist.pop(0)
len(citylist)
    

    

202

In [4]:
#get csv links for each city in dictionary 

cityURL = {}
baseurl = 'https://www.shazam.com/services/charts/csv/top-50/united-states/'
for city in citylist:
    csvurl = baseurl + str(city)
    cityURL[city] = csvurl

cityURL
    
#df = pd.read_csv('https://www.shazam.com/services/charts/csv/top-50/united-states/albany', skiprows = 2)

    

{'Aberdeen-SD': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Aberdeen-SD',
 'Aberdeen-WA': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Aberdeen-WA',
 'Abilene': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Abilene',
 'Akron': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Akron',
 'Albany': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Albany',
 'Albuquerque': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Albuquerque',
 'Allentown': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Allentown',
 'Anaheim': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Anaheim',
 'Anchorage': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Anchorage',
 'Asheville': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Asheville',
 'Atlanta': 'https://www.shazam.com/services/charts/csv/top-50/united-states/Atlanta',
 'Aurora': 'h

In [5]:
#Create Panda Dataframe
columnsPart = ["Song Name", "Artist"]
columns = columnsPart + citylist
dfCityTop20 = pd.DataFrame(columns = columns)
dfCityTop20

,Song Name,Artist,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma


In [6]:
# A Helper Function to fill in Song Rankings in the dfCityTop20 dataframe 

def updateSongRank(dfCity, SongIndex, dfCityTop20, SongLimit, CityName):
    
    #Append top 20 songs to the CityTop20 Dataframe 
    SongRank = 0
    while SongRank < SongLimit: 
        SongName =  dfCity.at[SongRank, 'Title']
        SongArtist =  dfCity.at[SongRank, 'Artist']
        
        # append new song/artist to dataframe if not already inside 
        if SongName not in dfCityTop20['Song Name'].unique():
            dfCityTop20 = dfCityTop20.append({"Song Name": SongName, "Artist": SongArtist}, ignore_index = True)
            global SongIndexCounter
            SongIndex.update({SongName: SongIndexCounter})  
            SongIndexCounter += 1
            #print(SongName)
            
        # fill in the value (rank) for the song for the specific city if not the same as previous day
        # update rank for next song
        if (dfCityTop20.at[SongIndex.get(SongName), CityName] != SongRank+1):
            dfCityTop20.at[SongIndex.get(SongName), CityName] = SongRank+1
        SongRank += 1
        
    #print(SongIndex)
    #Reset song rank
    return dfCityTop20

In [7]:

# Keeps track of the song and its index in the dfCityTop20 dataframe 
SongIndex = {}
SongIndexCounter = 0
SongLimit = 45


# Loop through each city 
for cityName in citylist: 
    cityurl = cityURL.get(cityName)
    dfCity = pd.read_csv(cityurl, skiprows = 2)
    print(dfCity, cityName)
    dfCityTop20 = updateSongRank(dfCity, SongIndex, dfCityTop20, SongLimit, cityName)
    
dfCityTop20

    Rank                                             Artist  \
0      1                                              Latto   
1      2                                      Glass Animals   
2      3                              Elton John & Dua Lipa   
3      4                                       The National   
4      5                                     Gerry Rafferty   
5      6                                         Steely Dan   
6      7                                           Måneskin   
7      8                                      Three 6 Mafia   
8      9                                       Cody Johnson   
9     10                                         Ed Sheeran   
10    11                                              GAYLE   
11    12                           Post Malone & The Weeknd   
12    13                                          Radiohead   
13    14                                        $uicideboy$   
14    15                   THE ANXIETY, WILLOW & Tyler 

    Rank                                   Artist  \
0      1                    Elton John & Dua Lipa   
1      2                 Post Malone & The Weeknd   
2      3                            Glass Animals   
3      4                                    Latto   
4      5                                Muni Long   
5      6                                    Gunna   
6      7                      Alesso & Katy Perry   
7      8                           Gerry Rafferty   
8      9  Bruno Mars, Anderson .Paak & Silk Sonic   
9     10     YoungBoy Never Broke Again & Birdman   
10    11                             Orville Peck   
11    12                            Three 6 Mafia   
12    13                                 Labrinth   
13    14                       Labrinth & Zendaya   
14    15         THE ANXIETY, WILLOW & Tyler Cole   
15    16                               Billy Swan   
16    17                                 Doja Cat   
17    18                                    GA

    Rank                                   Artist  \
0      1                    Elton John & Dua Lipa   
1      2                                    GAYLE   
2      3                           Brandi Carlile   
3      4                                 Jnr Choi   
4      5                                    Latto   
5      6                            Billie Eilish   
6      7                             Orville Peck   
7      8                           Gerry Rafferty   
8      9                                 Labrinth   
9     10                       Labrinth & Zendaya   
10    11                                    Adele   
11    12                                Muni Long   
12    13                                Lil Nas X   
13    14                                      SZA   
14    15         NEIKED & Mae Muller Feat. Polo G   
15    16                              Kodak Black   
16    17                            Gunna & Drake   
17    18                 Fireboy DML & Ed Shee

    Rank                                   Artist  \
0      1                    Elton John & Dua Lipa   
1      2                                    Latto   
2      3                                Muni Long   
3      4                                    GAYLE   
4      5                     Lauren Spencer-Smith   
5      6                              Kodak Black   
6      7                                 Jnr Choi   
7      8                                      SZA   
8      9                           Brandi Carlile   
9     10                                    Adele   
10    11         THE ANXIETY, WILLOW & Tyler Cole   
11    12                               Ed Sheeran   
12    13                             Moneybagg Yo   
13    14                 Fireboy DML & Ed Sheeran   
14    15         NEIKED & Mae Muller Feat. Polo G   
15    16                      Alesso & Katy Perry   
16    17                           Gerry Rafferty   
17    18                                  Farr

    Rank                                             Artist  \
0      1                              Elton John & Dua Lipa   
1      2                                              Latto   
2      3                                              GAYLE   
3      4                                             GIVĒON   
4      5                                      Billie Eilish   
5      6                                      Glass Animals   
6      7                                              Adele   
7      8                          Scarlett Johansson & Bono   
8      9                                     Gerry Rafferty   
9     10                                       Orville Peck   
10    11                   NEIKED & Mae Muller Feat. Polo G   
11    12                                Alesso & Katy Perry   
12    13                                         Pink Floyd   
13    14                                        Colter Wall   
14    15                                         Paul S

...

    Rank                                        Artist  \
0      1                                 Billie Eilish   
1      2                                     Muni Long   
2      3                                Gerry Rafferty   
3      4                         Elton John & Dua Lipa   
4      5                                  Orville Peck   
5      6                      Fireboy DML & Ed Sheeran   
6      7                                   Kodak Black   
7      8                                  Moneybagg Yo   
8      9                                      Jnr Choi   
9     10                                         Latto   
10    11                           Alesso & Katy Perry   
11    12  Rick Ross Feat. Jazmine Sullivan & 21 Savage   
12    13                                           SZA   
13    14                                 Glass Animals   
14    15                                    Ari Lennox   
15    16       Bruno Mars, Anderson .Paak & Silk Sonic   
16    17      

    Rank                                   Artist  \
0      1                                     Fun.   
1      2                          Honeymoon Suite   
2      3                            Steve Winwood   
3      4                                    AC/DC   
4      5                              The Effects   
5      6                             The Outfield   
6      7                           Carpenter Brut   
7      8                       Dave Matthews Band   
8      9                                Intocable   
9     10                          Olivier Raymond   
10    11                             Phil Collins   
11    12                               J.T. Music   
12    13                                 Chabelos   
13    14                             Garth Brooks   
14    15                              Super Lamas   
15    16                            Trevor Morris   
16    17                        Rex Orange County   
17    18                                      

    Rank                                       Artist  \
0      1                        Elton John & Dua Lipa   
1      2                                     Los Lara   
2      3                    Scarlett Johansson & Bono   
3      4                              Sebastián Yatra   
4      5                                Glass Animals   
5      6                                        Latto   
6      7                               Gerry Rafferty   
7      8             THE ANXIETY, WILLOW & Tyler Cole   
8      9                                   Ed Sheeran   
9     10                                    Lil Nas X   
10    11                                    Muni Long   
11    12                            Feni Feat. Yoflem   
12    13                                        Adele   
13    14                                 Orville Peck   
14    15      Bruno Mars, Anderson .Paak & Silk Sonic   
15    16                                Three 6 Mafia   
16    17                   Gesa

    Rank                                   Artist  \
0      1                    Elton John & Dua Lipa   
1      2                                    Latto   
2      3                            Glass Animals   
3      4                                  Farruko   
4      5                                    GAYLE   
5      6                             Orville Peck   
6      7                            Billie Eilish   
7      8         THE ANXIETY, WILLOW & Tyler Cole   
8      9                    Adolescentes Orquesta   
9     10         NEIKED & Mae Muller Feat. Polo G   
10    11                           Jessica Darrow   
11    12         Blxst & Tyga Feat. Ty Dolla $ign   
12    13                 CKay Feat. Axel & Dj Yo!   
13    14      Drake Feat. 21 Savage & Project Pat   
14    15                      Alesso & Katy Perry   
15    16                            Gunna & Drake   
16    17             Creedence Clearwater Revival   
17    18                  Eyedress Feat. Dent 

,Song Name,Artist,Aberdeen-SD,Aberdeen-WA,Abilene,Akron,Albany,Albuquerque,Allentown,Anaheim,...,Virginia-Beach,Warren-OH,Washington-D.C.,Wichita,Wilmington,Woonsocket,Worcester,Yakima,Youngstown,Yuma
0,Big Energy,Latto,1,1,4,3,5,2,2,1,...,3,4,10,5,3,NaN,6,6,12,2
1,Heat Waves,Glass Animals,2,NaN,3,10,30,3,29,6,...,23,10,14,3,11,NaN,14,5,24,3
2,Cold Heart (PNAU Remix),Elton John & Dua Lipa,3,NaN,1,1,1,1,1,2,...,5,13,4,1,2,NaN,1,1,1,1
3,I Should Live In Salt,The National,4,NaN,NaN,NaN,43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,NaN,NaN
4,Right Down The Line,Gerry Rafferty,5,NaN,8,36,8,12,17,4,...,26,37,3,24,30,NaN,5,7,36,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,Fair Trade [Mixed],Drake Feat. Travis Scott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN
1022,Me Meti En El Ruedo (En Vivo),Luis R Conriquez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,NaN,32
1023,Alien Blues,Vundabar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
1024,late at night,Roddy Ricch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34


In [8]:
dfCityTop20.to_csv('/Users/zacharywong/github/zacharywong2023/AtlanticRecords/IntermediateDataFrames/50CityDataTest.csv', index=False)